# Prédiction de l’achalandage des stations Bixi

Ce projet vise à prédire le taux d’entrées et de sorties par tranche horaire dans les stations Bixi à l’aide de plusieurs modèles :

- **STGNN (Spatio-Temporal Graph Neural Network)** : réseau neuronal à graphes spatio-temporels, stations comme nœuds, arêtes basées sur la distance ou le temps de déplacement.
- **LSTM (Long Short-Term Memory)** : réseau neuronal récurrent adapté aux séries temporelles.
- **Modèle statistique** : régression ou analyse classique pour servir de baseline.

Nous comparerons la performance de ces modèles sur les données d’achalandage.

---

## 1. Importation des librairies et des données

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Pour apprentissage
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
# Pour LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
# Pour le modèle statistique
import statsmodels.api as sm
# Pour le graphe
import networkx as nx

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "c:\Users\olidr\anaconda3\python.exe"
  * The NumPy version is: "1.24.3"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


## 2. Exploration et préparation des données

- Nettoyage et visualisation

- Construction du graphe des stations

## 3. Modélisation

### 3.1 Modèle statistique (baseline)

### 3.2 Modèle LSTM

### 3.3 Modèle STGNN (Spatio-Temporal Graph Neural Network)

## 4. Comparaison des performances

- Métriques : RMSE, MAE, etc.
- Visualisation des résultats